In [1]:
from pyhmy import cli
import json
import pandas as pd
import numpy as np
import os
import time

In [9]:
env = cli.download()
cli.environment.update(env)

Saved harmony binary to: `/Users/yishuang/Desktop/log_analysis/notebooks/staking_dashboard/bin/hmy`


In [10]:
path = os.getcwd() + "/bin/hmy"
cli.set_binary(path)

True

In [11]:
validator = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator all"))['result']   

In [12]:
i = validator[0]
info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']

In [13]:
info

{'current-epoch-performance': {'current-epoch-signing-percent': {'current-epoch-blocks-left': 25,
   'current-epoch-signed': 13,
   'current-epoch-signing-percentage': '1.000000000000000000',
   'current-epoch-to-sign': 13}},
 'currently-in-committee': True,
 'epos-status': 'currently elected and signing enough blocks to be eligible for election next epoch',
 'lifetime': {'apr': '0.000000000000000000',
  'blocks': {'signed': 15592, 'to-sign': 15592},
  'reward-accumulated': 3.3739381997601907e+22},
 'metrics': {'by-shard': [{'bls-public-key': 'afc4d81ab636323b04893c23d3404f38e95e9291b3d5299dade3d03822edd3a2b799bd6d64265a9a9cac025d5dbd600c',
    'earning-account': 'one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prh',
    'effective-stake': '212500000000000000000000.000000000000000000',
    'group-percent': '0.036349640780020527',
    'overall-percent': '0.011631885049606569',
    'shard-id': 0}],
  'total-effective-stake': '212500000000000000000000.000000000000000000'},
 'total-delegation': 1e+

In [5]:
shard = []
availability = []
name = []
epos_status = []
committee = []
website = []
details = []
address = []
for i in validator:  
    info = json.loads(cli.single_call("hmy --node=https://api.s0.os.hmny.io blockchain validator information " + i))['result']
    if not info['metrics']:
        shard.append(np.nan)
    else:
        shard.append(info['metrics']['by-shard'][0]['shard-id'])
    if not info['current-epoch-performance']:
        availability.append(np.nan)
    else:
        availability.append(info['current-epoch-performance']['current-epoch-signing-percent'])
    name.append(info['validator']['name'])
    epos_status.append(info['epos-status'])
    committee.append(info['currently-in-committee'])
    website.append(info['validator']['website'])
    details.append(info['validator']['details'])
    address.append(info['validator']['address'])


In [6]:
df = pd.DataFrame(list(zip(shard, availability, name, epos_status, committee, website, details, address)), \
                  columns =['shard-id', 'availability', 'name','epos-status', 'currently-in-committee', \
                            'website','details','address'])
df = pd.concat([df.drop(['availability'], axis=1), df['availability'].apply(pd.Series)], axis=1)
df.drop([0], axis = 1, inplace = True)
df = df.sort_values(by=['shard-id'])
df.reset_index(inplace = True, drop = True)

In [7]:
df

,shard-id,name,epos-status,currently-in-committee,website,details,address,current-epoch-blocks-left,current-epoch-signed,current-epoch-signing-percentage,current-epoch-to-sign
0,0.0,OSTNSentry0,currently elected and signing enough blocks to...,True,sentry@harmony.one,OSTN Sentry for Shard 0 t3.small,one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prh,5,33.0,1.000000000000000000,33.0
1,0.0,theRealONE,currently elected and signing enough blocks to...,True,NA,@BaJaSam validator node,one1ekup98s5tqxtr5hdzsz664cfy579jpq6w5smrr,4,34.0,1.000000000000000000,34.0
2,0.0,WeAreONE,currently elected and signing enough blocks to...,True,AKC12190.COM,AKCValidator,one17tj2jjehdlg8xfgp48xpeyqur2qf6nvs88jvyu,5,33.0,1.000000000000000000,33.0
3,0.0,WYLDKARD,currently elected and signing enough blocks to...,True,website url here,This is a validator in test in WEST coast,one1hwn68w7m5g8daltvrjkmq23qn58nrvr3yz6ryt,4,34.0,1.000000000000000000,34.0
4,1.0,OSTNSentry1,currently elected and signing enough blocks to...,True,sentry@harmony.one,OSTN Sentry for Shard 1 t3.small,one10fjqteq6q75nm62cx8vejqsk7mc8t5hle8ewnl,4,34.0,1.000000000000000000,34.0
5,1.0,Gordo,currently elected and signing enough blocks to...,True,https://gordo.pt,Gordo validating,one1mpzx5wr2kmz9nvkhsgj6jr6zs87ahm0gxmhlck,8,23.0,0.766666666666666667,30.0
6,1.0,JohnV1,currently elected and signing enough blocks to...,True,john@harmony.one,John Validator t3.small,one1y5n7p8a845v96xyx2gh75wn5eyhtw5002lah27,4,34.0,1.000000000000000000,34.0
7,1.0,Yelllowsin_1-OpenStaking #P-OPS,currently elected and signing enough blocks to...,True,Yelllowsin_1,Yelllowsin_1,one170xqsfzm4xdmuyax54t5pvtp5l5yt66u50ctrp,3,35.0,1.000000000000000000,35.0
8,1.0,SK4T3R|ST4K3R,currently elected and signing enough blocks to...,True,http://bit.ly/dillagr,TGT:4-9S,one17d60e5nvjnzechwl56y38ze6w49wejhtanncra,4,34.0,1.000000000000000000,34.0
9,1.0,Catana,currently elected and signing enough blocks to...,True,onevalidator.com,SoftNode,one19cuygr776f7j9ep97hjm0np9ay6nus9w5msy0n,4,31.0,0.911764705882352941,34.0


In [8]:
df.to_csv("../../csv/validator.tsv", index=False)